## Imports

In [11]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd

from pandas_geojson import to_geojson, write_geojson
from geopy.geocoders import GoogleV3
from textwrap import shorten

## Read-in

In [12]:
df = pd.read_csv('South Florida architects map - Sheet1.csv')

In [13]:
df.columns

Index(['full_address', 'architects', 'developers', 'description',
       'card_description', 'story_link'],
      dtype='object')

## Google Maps API Geolocater Setup

In [14]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [15]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['full_address'].apply(geocode)

In [16]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Article Link Formatter

In [17]:
df.columns

Index(['full_address', 'architects', 'developers', 'description',
       'card_description', 'story_link', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [18]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2023/11...
1    <a href="https://therealdeal.com/miami/2021/02...
2    <a href="https://therealdeal.com/miami/2023/12...
3    <a href="https://therealdeal.com/miami/2023/12...
4    <a href="https://therealdeal.com/miami/2023/12...
5    <a href="https://therealdeal.com/miami/2022/10...
6    <a href="https://therealdeal.com/miami/2023/07...
7    <a href="https://therealdeal.com/miami/2018/08...
8    <a href="https://therealdeal.com/miami/2023/09...
9    <a href="https://therealdeal.com/miami/2023/07...
Name: description_link, dtype: object

## Write to GeoJSON

In [21]:
df.columns

Index(['full_address', 'architects', 'developers', 'description',
       'card_description', 'story_link', 'geocoded', 'lat', 'lon',
       'short_description', 'remaining_desc', 'description_link'],
      dtype='object')

In [27]:
geo_json = to_geojson(df=df, lat='lat', lon='lon',
                     properties=['full_address','description','architects','developers'])
write_geojson(geo_json, filename='SoFlaArchitects.geojson', indent=4)

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [25]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/So_Fla_Architects'

In [26]:
print(base_name)

https://trd-digital.github.io/trd-news-interactive-maps/So_Fla_Architects
